In [ ]:
#default_exp synthesizer

# Synthesizer
> Loading, defining and training wavenet based speech synthesizers from various audio representations.

# Imports

In [ ]:
from fastai2.basics import *
from fastai2.callback.all import *
from MiniAudio.data import *
from MiniAudio.layers import *
from MiniAudio.transforms import *

In [ ]:
import librosa
import librosa.display
import IPython

In [ ]:
file = '/home/cerberus/.fastai/data/es/clips/'

# DataLoaders

In [ ]:
data = DataBlock((SpecBlock(librosa.cqt), SignalBlock()),
                 get_items=get_audio_files,
                 get_x = Pipeline([AudioLoader(), Clipper(220500), NormalizeAudio(), ToTensor()]),
                 get_y = Pipeline([AudioLoader(), Clipper(220500), NormalizeAudio(), ToTensor(), MuLawEncoding(256)]),
                 splitter=RandomSplitter())
dls = data.dataloaders(file, bs=1, num_workers=0)

Could not do one pass in your dataloader, there is something wrong in it


# Training

In [ ]:
model = WaveNet(84, 256, 10, 3, 44100, 5)

In [ ]:
learner = Learner(dls, model, loss_func=L1LossFlat())

In [ ]:
learner.

In [ ]:
learner.fit_one_cycle(4)

In [ ]:
learner.predict(TensorSpec(b[0].unsqueeze(0)))

# Results